# Train

In [1]:
"""For visi_depth
"""
# !CUDA_VISIBLE_DEVICES=3 python train.py --split eigen_zhou \
#                                             --load_weights_folder './history/model_22_09/models/weights_8' \
#                                             --png --backbone mobile_net_v3 \
#                                             --depth_decoder hr_depth \
#                                             --predict_visibility \
#                                             --model_name "model_24_09" \
#                                             --batch_size 6 \
#                                             --num_workers 4 \
#                                             --num_epochs 8 \
#                                             --log_frequency 3600 \
#                                             --start_saving_weight 1

In [35]:
!nvidia-smi

Thu Oct  6 01:54:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   27C    P0    52W / 300W |  14822MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   28C    P0    56W / 300W |   3184MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [37]:
!CUDA_VISIBLE_DEVICES=1 python train-v2.py --split eigen_zhou \
                                            --load_weights_folder "history/the_last/models/weights_9" \
                                            --start_epoch 10 \
                                            --png --backbone resnet \
                                            --depth_decoder hr_depth \
                                            --depth_supervised_loss \
                                            --supervised_loss_weight 0.1 \
                                            --model_name "the_last" \
                                            --batch_size 6 \
                                            --num_workers 6 \
                                            --num_epochs 10 \
                                            --start_saving_weight 1 

Version trainer_v3
Back bone:resnet - Depth decoder:hr_depth
Use depth supervised loss
loading model from folder history/the_last/models/weights_9
Loading encoder weights...
Loading depth weights...
Loading pose_encoder weights...
Loading pose weights...
Loading Adam weights
Training model named:
   the_last
Models and tensorboard events files are saved to:
   history
Training is using:
   cuda
/root/miniconda3/envs/mm-cuda11.1-torch1.9/lib/python3.7/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
Using split:
   eigen_zhou
There are 39810 training items and 4424 validation items

/root/miniconda3/envs/mm-cuda11.1-torch1.9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 

In [31]:
!CUDA_VISIBLE_DEVICES=1 python train-v2.py --split eigen_zhou \
                                            --start_epoch 0 \
                                            --png --backbone mobile_net_v3 \
                                            --depth_decoder hr_depth \
                                            --depth_supervised_loss \
                                            --supervised_loss_weight 0.2 \
                                            --model_name "semi_sup_mobile" \
                                            --batch_size 6 \
                                            --num_workers 5 \
                                            --scheduler_step_size 5 \
                                            --num_epochs 12 \
                                            --start_saving_weight 1 \
                                            --make_it_quickly 2

Version trainer_v3
Back bone:mobile_net_v3 - Depth decoder:hr_depth
Use depth supervised loss
Training model named:
   semi_sup_mobile
Models and tensorboard events files are saved to:
   history
Training is using:
   cuda
/root/miniconda3/envs/mm-cuda11.1-torch1.9/lib/python3.7/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
Using split:
   eigen_zhou
There are 19905 training items and 4424 validation items

/root/miniconda3/envs/mm-cuda11.1-torch1.9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of 

In [23]:
"""Test sup weight
"""
!CUDA_VISIBLE_DEVICES=1 python train-v2.py --split eigen_zhou \
                                            --start_epoch 0 \
                                            --png --backbone resnet \
                                            --depth_decoder hr_depth \
                                            --model_name "testing" \
                                            --batch_size 6 \
                                            --num_workers 6 \
                                            --num_epochs 16 \
                                            --start_saving_weight 1 \

Version trainer_v3
Back bone:resnet - Depth decoder:hr_depth
Training model named:
   testing
Models and tensorboard events files are saved to:
   history
Training is using:
   cuda
/root/miniconda3/envs/mm-cuda11.1-torch1.9/lib/python3.7/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
Using split:
   eigen_zhou
There are 9952 training items and 4424 validation items

/root/miniconda3/envs/mm-cuda11.1-torch1.9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more detai

## Import library

In [ ]:
!nvidia-smi

In [2]:
import numpy as np
import time
import os
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from test_simple import test_simple
from utils import *
from kitti_utils import *
from layers import *
import datasets
import networks
from trainer_v2 import Trainer

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
class Opt:
    def __init__(self):
        # path
        self.data_path = './kitti_data'
        self.log_dir = './history/staff'
        self.load_weights_folder = './history/mobile_hr/models/weights_15'
        self.models_to_load = ["encoder", "depth", "pose_encoder", "pose"]
        self.weights_init = True
        
        # training
        self.depth_decoder = 'hr_depth'
        self.backbone = 'mobile_net_v3'
        self.model_name = ''
        self.predict_visibility = True
        self.split = 'eigen_zhou'
        
        self.num_layers = 18
        self.dataset = 'kitti'
        self.png = True
        self.height = 192
        self.width = 640
        self.scales = [0,1,2,3]
        self.frame_ids = [0,1]
        self.min_depth = 0.1
        self.max_depth = 100
        self.disparity_smoothness = 1e-3
        self.no_cuda = False
        
        # evaluation
        self.eval_split = 'eigen'
        self.splits_dir = './splits'
        self.eval_show = True
        
        # optimization
        self.batch_size = 2
        self.num_workers = 1
        self.learning_rate = 1e-4
        self.num_epochs = 5
        self.scheduler_step_size = 15

        # ablation
        self.avg_reprojection = False
        self.disable_automasking = False
        self.predictive_mask = False
        self.no_ssim = False
        self.pose_model_input = "pairs"
        self.use_stereo = False
        self.use_visibility = False
        self.v1_multiscale = False
        self.pose_model_type = "separate_resnet"
opt = Opt()

## Main

In [5]:
train_class = Trainer(opt)

Back bone:mobile_net_v3 - Depth decoder:hr_depth
loading model from folder ./history/mobile_hr/models/weights_15
Loading encoder weights...
Loading depth weights...
Loading pose_encoder weights...
Loading pose weights...
Training model named:
   
Models and tensorboard events files are saved to:
   ./history/staff
Training is using:
   cuda


/root/miniconda3/envs/mm-cuda11.1-torch1.9/lib/python3.7/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Using split:
   eigen_zhou
There are 39810 training items and 4424 validation items



In [6]:
inputs = next(iter(train_class.train_loader))
for key, ipt in inputs.items():
    if type(ipt) is list:
        continue
    inputs[key] = ipt.to(train_class.device)
features = train_class.models['encoder'](inputs['color_aug',0,0])
for f in features:
    print(f'{f.shape}')
outputs, losses = train_class.process_batch(inputs)

torch.Size([2, 16, 96, 320])
torch.Size([2, 24, 48, 160])
torch.Size([2, 40, 24, 80])
torch.Size([2, 80, 12, 40])
torch.Size([2, 160, 6, 20])


/root/miniconda3/envs/mm-cuda11.1-torch1.9/lib/python3.7/site-packages/torch/nn/functional.py:4004: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "


## Something

In [23]:
features = train_class.models['encoder'](inputs['color_aug',0,0])
outputs = train_class.models['depth'](features)
outputs[('disp',0)].shape

torch.Size([2, 1, 192, 640])

In [18]:
train_class.models['visibility_net'] = train_class.models['visibility_net'].to(train_class.device)
preds = train_class.models['visibility_net'](features)
gts = torch.max(inputs['depth_gt'].view(inputs['depth_gt'].shape[0], inputs['depth_gt'].shape[1], -1), dim = 2).values

In [29]:

min_depth = torch.Tensor([0.01, 0.01]).reshape(-1,1,1).to(train_class.device)
disp_to_depth(outputs[('disp',0)], min_depth, max_depth)[0].shape

torch.Size([2, 2, 192, 640])

In [32]:
max_depth = preds.reshape(-1,1,1,1)
(outputs[('disp',0)] / max_depth).shape

torch.Size([2, 1, 192, 640])

In [28]:
visibility_loss = nn.MSELoss()
visibility_loss(preds, gts)

tensor(6210.2197, device='cuda:0', grad_fn=<MseLossBackward0>)

In [46]:
a = np.random.rand(2,1,2,2)
print(a)
b = np.array([2,3]).reshape(-1,1,1,1)
print(b.shape)
print(a/b)

[[[[0.97784222 0.30816645]
   [0.01524218 0.91396792]]]


 [[[0.44496342 0.72753946]
   [0.00526706 0.15152851]]]]
(2, 1, 1, 1)
[[[[0.48892111 0.15408322]
   [0.00762109 0.45698396]]]


 [[[0.14832114 0.24251315]
   [0.00175569 0.0505095 ]]]]


In [44]:
a[0][0][0]

array([0.2873559 , 0.76961584])